[![Buy Me a cofee](https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/coffe4.gif?raw=true ) ](https://www.buymeacoffee.com/wilzamguerrero)

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/save.png?raw=true' height="50" alt="netdata"/></center>
import os
from IPython.display import clear_output

save_models_dir = "/content/sample_data"#@param {type:"string"}
os.chdir(save_models_dir)
url = "https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt" #@param {type:"string"}
!wget --no-check-certificate $url

clear_output()
print('[1;96mModel Save ✓')

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/drive.png?raw=true' height="50" alt="netdata"/></center>
from IPython.display import clear_output
from google.colab import drive
drive.mount('/content/drive')
clear_output()
print('[1;96mDrive Connected ✓')

In [10]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v13i</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/sample_data" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "V2.1" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"
model_init = "" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = False #@param {type:"boolean"}
SDZ_Parameterization_V = False #@param {type:"boolean"}
SDZ_Big_Model= False #@param {type:"boolean"}
SDZ_Continue = False #@param {type:"boolean"}

if SDZ_Big_Model:
  !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' $SDZ_webui_path'/modules/shared.py'
else:
  !sed -i 's@cmd_opts.lowram else \"cuda\"@cmd_opts.lowram else \"cpu\"@' $SDZ_webui_path'/modules/shared.py'

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    #!wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
     
    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
   
    %cd ..

    if SDZ_Parameterization_V:
      os.chdir(SDZ_webui_path)
      !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
      with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
        filedata = file.read()
        filedata = filedata.replace('parameterization: "v"','#sdz')
      with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
        file.write(filedata)
    else:
      os.chdir(SDZ_webui_path)
      !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

      if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        %cd extensions
        # Install Extensions 
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

if SDZ_Continue:
  if not os.path.exists('/usr/local/conda-meta/'):
    with capture.capture_output() as cap:
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        if SDZ_Parameterization_V:
          os.chdir(SDZ_webui_path)
          !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
            filedata = file.read()
            filedata = filedata.replace('parameterization: "v"','#sdz')
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
            file.write(filedata)
        else:
          os.chdir(SDZ_webui_path)
          !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml

        !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

  if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        if SDZ_Parameterization_V:
          os.chdir(SDZ_webui_path)
          !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
            filedata = file.read()
            filedata = filedata.replace('parameterization: "v"','#sdz')
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
            file.write(filedata)
        else:
          os.chdir(SDZ_webui_path)
          !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml

          !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')
        if SDZ_Version == "V1.5":
          !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
        elif SDZ_Version == "V2.1":
          !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae


Installing SDZ ✓
Preparing SDZ System ✓
Python 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]
Commit hash: 685f9631b56ff8bd43bce24ff5ce0f9a0e9af490
Cloning Stable Diffusion into repositories/stable-diffusion-stability-ai...
Cloning Taming Transformers into repositories/taming-transformers...
Cloning K-diffusion into repositories/k-diffusion...
Cloning CodeFormer into repositories/CodeFormer...
Cloning BLIP into repositories/BLIP...
Installing requirements for Web UI
Launching Web UI with arguments: --share --api --config v2-inference-v.yaml --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir= --ckpt= --theme=dark --disable-safe-unpickle --no-half-vae
Checkpoint in --ckpt argument not found (Possible it was moved to /content/sample_data/stable-diffusion-webui/models/Stable-diffusion: 
Checkpoint v1-5-fp16.ckpt [a2a802b2] not found; loading fallback content/sample_data/stable-diffusion-webui/models/Stable-diffusion/v21.ckpt [6bccbcc6]
LatentDiffusion: R

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v13h</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"
model_init = "/content/drive/Shareddrives/Data13/zdream/MyDrive/zmarluzz2_2028.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = False #@param {type:"boolean"}
SDZ_Parameterization_V = False #@param {type:"boolean"}
SDZ_Big_Model= False #@param {type:"boolean"}
SDZ_Continue = False #@param {type:"boolean"}

if SDZ_Big_Model:
  !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' $SDZ_webui_path'/modules/shared.py'
else:
  !sed -i 's@cmd_opts.lowram else \"cuda\"@cmd_opts.lowram else \"cpu\"@' $SDZ_webui_path'/modules/shared.py'

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    #!wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget -N https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget -N https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
     
    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
   
    %cd ..

    if SDZ_Parameterization_V:
      os.chdir(SDZ_webui_path)
      !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
      with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
        filedata = file.read()
        filedata = filedata.replace('parameterization: "v"','#sdz')
      with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
        file.write(filedata)
    else:
      os.chdir(SDZ_webui_path)
      !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

      if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        %cd extensions
        # Install Extensions 
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

if SDZ_Continue:
  if not os.path.exists('/usr/local/conda-meta/'):
    with capture.capture_output() as cap:
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        if SDZ_Parameterization_V:
          os.chdir(SDZ_webui_path)
          !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
            filedata = file.read()
            filedata = filedata.replace('parameterization: "v"','#sdz')
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
            file.write(filedata)
        else:
          os.chdir(SDZ_webui_path)
          !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml

        !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

  if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        if SDZ_Parameterization_V:
          os.chdir(SDZ_webui_path)
          !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
            filedata = file.read()
            filedata = filedata.replace('parameterization: "v"','#sdz')
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
            file.write(filedata)
        else:
          os.chdir(SDZ_webui_path)
          !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml

          !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

        if SDZ_Version == "V1.5":
          !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle 
        elif SDZ_Version == "V2.1":
          !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle 


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v13g</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"
model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/v2-1_768-ema-pruned.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = False #@param {type:"boolean"}
SDZ_Parameterization_V = True #@param {type:"boolean"}
SDZ_Big_Model= True #@param {type:"boolean"}
SDZ_Continue = True #@param {type:"boolean"}

if SDZ_Big_Model:
  !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' $SDZ_webui_path'/modules/shared.py'
else:
  !sed -i 's@cmd_opts.lowram else \"cuda\"@cmd_opts.lowram else \"cpu\"@' $SDZ_webui_path'/modules/shared.py'

if SDZ_Parameterization_V:
  os.chdir(SDZ_webui_path)
  !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
  with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
    filedata = file.read()
    filedata = filedata.replace('parameterization: "v"','#sdz')
  with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
    file.write(filedata)
else:
  os.chdir(SDZ_webui_path)
  !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    #!wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget -N https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget -N https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
     
    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
   
    %cd ..

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

      if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        %cd extensions
        # Install Extensions 
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

if SDZ_Continue:
  if not os.path.exists('/usr/local/conda-meta/'):
    with capture.capture_output() as cap:
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

  if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

        if SDZ_Version == "V1.5":
          !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle 
        elif SDZ_Version == "V2.1":
          !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle 


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v13g</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"
model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/v2-1_768-ema-pruned.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = False #@param {type:"boolean"}
SDZ_Continue = True #@param {type:"boolean"}
SDZ_Parameterization_V = True #@param {type:"boolean"}
Large_Model= True #@param {type:"boolean"}

if Large_Model:
  !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' $SDZ_webui_path'/modules/shared.py'
else:
  !sed -i 's@cmd_opts.lowram else \"cuda\"@cmd_opts.lowram else \"cpu\"@' $SDZ_webui_path'/modules/shared.py'

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget -N https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget -N https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
     
    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
    
    # fix inference V2
    if SDZ_Parameterization_V:
      with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
        filedata = file.read()
        filedata = filedata.replace('parameterization: "v"','#sdz')
      with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)

    else:
      with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
        filedata = file.read()
      filedata = filedata.replace('#sdz','parameterization: "v"')
      with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
        file.write(filedata)

    %cd ..

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

      if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        # fix inference V2
        if SDZ_Parameterization_V:
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
            filedata = file.read()
            filedata = filedata.replace('parameterization: "v"','#sdz')
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
              file.write(filedata)

        else:
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
            filedata = file.read()
          filedata = filedata.replace('#sdz','parameterization: "v"')
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
            file.write(filedata)
        %cd extensions
        # Install Extensions 
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

if SDZ_Continue:
  if not os.path.exists('/usr/local/conda-meta/'):
    with capture.capture_output() as cap:
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

        # fix inference V2
        if SDZ_Parameterization_V:
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
            filedata = file.read()
            filedata = filedata.replace('parameterization: "v"','#sdz')
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
              file.write(filedata)

        else:
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
            filedata = file.read()
          filedata = filedata.replace('#sdz','parameterization: "v"')
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
            file.write(filedata)

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

  if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        # fix inference V2
        if SDZ_Parameterization_V:
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
            filedata = file.read()
            filedata = filedata.replace('parameterization: "v"','#sdz')
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
              file.write(filedata)

        else:
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
            filedata = file.read()
          filedata = filedata.replace('#sdz','parameterization: "v"')
          with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
            file.write(filedata)

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

        if SDZ_Version == "V1.5":
          !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle 
        elif SDZ_Version == "V2.1":
          !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle 


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v13f0</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"
model_init = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz/stable-diffusion-webui/zmarluzz2_2028.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = False #@param {type:"boolean"}
SDZ_Parameterization_V = True #@param {type:"boolean"}
SDZ_Continue = True #@param {type:"boolean"}
Large_Model= True #@param {type:"boolean"}


if Large_Model:
  !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' $SDZ_webui_path'/modules/shared.py'
else:
  !sed -i 's@cmd_opts.lowram else \"cuda\"@cmd_opts.lowram else \"cpu\"@' $SDZ_webui_path'/modules/shared.py'

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
     
    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
    %cd ..

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

      if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        %cd extensions
        # Install Extensions 
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

if SDZ_Continue:
  if not os.path.exists('/usr/local/conda-meta/'):
    with capture.capture_output() as cap:
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

  if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

        if SDZ_Version == "V1.5":
          !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle 
        elif SDZ_Version == "V2.1":
          !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle 


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v13f</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"
model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/v2-1_768-ema-pruned.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = False #@param {type:"boolean"}
SDZ_Continue = True #@param {type:"boolean"}

Large_Model= True #@param {type:"boolean"}

if Large_Model:
  !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' $SDZ_webui_path'/modules/shared.py'
else:
  !sed -i 's@cmd_opts.lowram else \"cuda\"@cmd_opts.lowram else \"cpu\"@' $SDZ_webui_path'/modules/shared.py'

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
     
    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
          # fix Dreambooth v21
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
      filedata = file.read()
    filedata = filedata.replace('parameterization: "v"','')
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
      file.write(filedata)
    %cd ..

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

      if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)
        %cd extensions
        # Install Extensions 
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

if SDZ_Continue:
  if not os.path.exists('/usr/local/conda-meta/'):
    with capture.capture_output() as cap:
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

  if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

        if SDZ_Version == "V1.5":
          !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle 
        elif SDZ_Version == "V2.1":
          !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle 


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v13f</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"
model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/v2-1_768-ema-pruned.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = False #@param {type:"boolean"}
SDZ_Continue = True #@param {type:"boolean"}

Large_Model= True #@param {type:"boolean"}
#@markdown  - Check if you have trouble loading a model 7GB+

if Large_Model:
  !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/shared.py
else:
  !sed -i 's@cmd_opts.lowram else \"cuda\"@cmd_opts.lowram else \"cpu\"@' /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/shared.py

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
     
    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
          # fix Dreambooth v21
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
      filedata = file.read()
    filedata = filedata.replace('parameterization: "v"','')
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
      file.write(filedata)
    %cd ..

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

      if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)
        %cd extensions
        # Install Extensions 
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

if SDZ_Continue:
  if not os.path.exists('/usr/local/conda-meta/'):
    with capture.capture_output() as cap:
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

  if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

        if SDZ_Version == "V1.5":
          !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle 
        elif SDZ_Version == "V2.1":
          !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle 


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v13e</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"

model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/Zv2-1_768-fp16.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = False #@param {type:"boolean"}
SDZ_Continue = True #@param {type:"boolean"}

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
     
    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
          # fix Dreambooth v21
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
      filedata = file.read()
    filedata = filedata.replace('parameterization: "v"','')
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
      file.write(filedata)
    %cd ..

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

      if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)
        %cd extensions
        # Install Extensions 
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

if SDZ_Continue:
  if not os.path.exists('/usr/local/conda-meta/'):
    with capture.capture_output() as cap:
        os.chdir(SDZ_webui_path)
        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

  if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)

        if SDZ_Version == "V1.5":
          !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
        elif SDZ_Version == "V2.1":
          !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v13d</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"

model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/Zv2-1_768-fp16.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = False #@param {type:"boolean"}
SDZ_Continue = True #@param {type:"boolean"}

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
     
    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
          # fix Dreambooth v21
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
      filedata = file.read()
    filedata = filedata.replace('parameterization: "v"','')
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
      file.write(filedata)
    %cd ..

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

      if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)
        %cd extensions
        # Install Extensions 
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

if SDZ_Continue:
  if not os.path.exists('/usr/local/conda-meta/'):
    with capture.capture_output() as cap:
        os.chdir(SDZ_webui_path)
        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

  if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)

        if SDZ_Version == "V1.5":
          !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
        elif SDZ_Version == "V2.1":
          !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v13c</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"

model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/Zv2-1_768-fp16.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = False #@param {type:"boolean"}
SDZ_Continue = True #@param {type:"boolean"}

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
     
    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
          # fix Dreambooth v21
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
      filedata = file.read()
    filedata = filedata.replace('parameterization: "v"','')
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
      file.write(filedata)
    %cd ..

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

      if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)
        %cd extensions
        # Install Extensions 
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

if SDZ_Continue:
  if not os.path.exists('/usr/local/conda-meta/'):
    with capture.capture_output() as cap:
        os.chdir(SDZ_webui_path)
        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')

  if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System Continue ✓')
        if SDZ_Version == "V1.5":
          !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
        elif SDZ_Version == "V2.1":
          !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v13b</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"

model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/Zv2-1_768-fp16.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = True #@param {type:"boolean"}
SDZ_Continue = False #@param {type:"boolean"}

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
     
    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
          # fix Dreambooth v21
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
      filedata = file.read()
    filedata = filedata.replace('parameterization: "v"','')
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
      file.write(filedata)
    %cd ..

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y

      if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)
        %cd extensions
        # Install Extensions 
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

if SDZ_Continue:
  if not os.path.exists('/usr/local/conda-meta/') = if not SDZ_Continue

  if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        clear_output()
        print('[1;90mPreparing SDZ System ✓')
        if SDZ_Version == "V1.5":
          !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
        elif SDZ_Version == "V2.1":
          !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae


In [ ]:
Podrías simplificar este código de Python

#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v13</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"

model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/Zv2-1_768-fp16.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = True #@param {type:"boolean"}
SDZ_Continue = False #@param {type:"boolean"}

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
     
    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
          # fix Dreambooth v21
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
      filedata = file.read()
    filedata = filedata.replace('parameterization: "v"','')
    with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
      file.write(filedata)
    %cd ..

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        #!wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        #!bash ./py310.sh -b -f -p /usr/local
        #!python -m ipykernel install --name "py310" --user
        #!conda install -c "xformers/label/dev" xformers -y

      if os.path.exists('/usr/local/conda-meta/'):
        os.chdir(SDZ_webui_path)
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
        # fix Dreambooth v21
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
          filedata = file.read()
          filedata = filedata.replace('parameterization: "v"','')
        with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
          file.write(filedata)
        %cd extensions
        # Install Extensions 
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

if SDZ_Continue:  
  os.chdir(SDZ_webui_path)
  clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ1.5.png?raw=true' height="50" alt="netdata"/></center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output

Model_Base_15 = False #@param {type:"boolean"}

SDZ = "stable-diffusion-webui"
SDZ_install = "/content/sample_data" #@param {type:"string"}
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)

model_init = "" #@param {type:"string"}
models_dir = "" #@param {type:"string"}

with capture.capture_output() as cap:

  if os.path.exists(SDZ):
    shutil.rmtree(SDZ)
  !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  if not os.path.exists(SDZ):
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  %cd stable-diffusion-webui/models/Stable-diffusion
  
  if Model_Base_15:
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt

    %cd ..
    %cd ..
    %cd stable-diffusion-webui

  clear_output()
print('[1;90mInstalling SDZ Xformers ✓')

with capture.capture_output() as cap:
  if not os.path.exists('/usr/local/conda-meta/'):
    !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
    !bash ./py310.sh -b -f -p /usr/local
    !python -m ipykernel install --name "py310" --user
    !conda install -c "xformers/label/dev" xformers -y

  clear_output()
print('[1;96mInstalling SDZ with Extentions ✓')

with capture.capture_output() as cap:
  if os.path.exists('/usr/local/conda-meta/'):

    # Install style - config
    os.chdir(SDZ_webui_path)
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    
    # Install Extensions
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git

    os.chdir(SDZ_install+"/"+SDZ)

  clear_output()
print('[1;90mPreparing SDZ System ✓')

!python launch.py --share --api --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --disable-safe-unpickle --no-half-vae --ckptfix


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.1.png?raw=true' height="50" alt="netdata"/></center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output

Model_Base_21 = False #@param {type:"boolean"}

SDZ = "stable-diffusion-webui"
SDZ_install = "/content/sample_data" #@param {type:"string"}
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)

model_init = "" #@param {type:"string"}
models_dir = "" #@param {type:"string"}

with capture.capture_output() as cap:

  if os.path.exists(SDZ):
    shutil.rmtree(SDZ)
  !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  if not os.path.exists(SDZ):
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  %cd stable-diffusion-webui/models/Stable-diffusion
  
  if Model_Base_21:
    !wget https://huggingface.co/uwg/modelz_base/blob/main/Zv2-1_768-fp16.ckpt

    %cd ..
    %cd ..
    %cd stable-diffusion-webui

  clear_output()
print('[1;90mInstalling SDZ Xformers ✓')

with capture.capture_output() as cap:
  if not os.path.exists('/usr/local/conda-meta/'):
    !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
    !bash ./py310.sh -b -f -p /usr/local
    !python -m ipykernel install --name "py310" --user
    !conda install -c "xformers/label/dev" xformers -y

  clear_output()
print('[1;96mInstalling SDZ with Extentions ✓')

with capture.capture_output() as cap:
  if os.path.exists('/usr/local/conda-meta/'):

    # Install style - config
    os.chdir(SDZ_webui_path)
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    
    # Install Extensions
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git

    os.chdir(SDZ_install+"/"+SDZ)

  clear_output()
print('[1;90mPreparing SDZ System ✓')

!python launch.py --share --api --config=v2-inference-v.yaml --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --disable-safe-unpickle --no-half-vae --ckptfix

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/></center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/sample_data" #@param {type:"string"}
SDZ_Version = "V1.5" #@param ["V1.5", "V2.1-768"]
Download_Base_Model = "V1.5" #@param ["", "V1.5", "V2.1-768"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)

model_init = "" #@param {type:"string"}
models_dir = "" #@param {type:"string"}

with capture.capture_output() as cap:

  if os.path.exists(SDZ):
    shutil.rmtree(SDZ)
  !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  if not os.path.exists(SDZ):
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  %cd stable-diffusion-webui/models/Stable-diffusion

  if Download_Base_Model == "V1.5":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt

  elif Download_Base_Model == "V2.1-768":
    !wget https://huggingface.co/uwg/modelz_base/blob/main/Zv2-1_768-fp16.ckpt

  else:
    Download_Base_Model == ""
  %cd ..
  %cd ..
  %cd stable-diffusion-webui

  clear_output()
print('[1;90mInstalling SDZ Xformers ✓')

with capture.capture_output() as cap:
  if not os.path.exists('/usr/local/conda-meta/'):
    !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
    !bash ./py310.sh -b -f -p /usr/local
    !python -m ipykernel install --name "py310" --user
    !conda install -c "xformers/label/dev" xformers -y
    
  if os.path.exists('/usr/local/conda-meta/'):

    # Install style - config
    os.chdir(SDZ_webui_path)
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml

    # Install Extensions
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git

    os.chdir(SDZ_install+"/"+SDZ)

  clear_output()
print('[1;90mPreparing SDZ System ✓')

if SDZ_Version == "V1.5":
  !python launch.py --share --api --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --disable-safe-unpickle --no-half-vae
elif SDZ_Version == "V2.1-768":
  !python launch.py --share --api --config=v2-inference-v.yaml --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --disable-safe-unpickle --no-half-vae --medvram


In [ ]:
!python launch.py --share --api --config=v2-inference-v.yaml --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --no-half

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/></center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/sample_data" #@param {type:"string"}
SDZ_Version = "V2.1-768" #@param ["V1.5", "V2.1-768"]
Download_Base_Model = "V2.1-768" #@param ["", "V1.5", "V2.1-768"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)

model_init = "" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = False #@param {type:"boolean"}

with capture.capture_output() as cap:

  if os.path.exists(SDZ):
    shutil.rmtree(SDZ)
  !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  if not os.path.exists(SDZ):
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  %cd stable-diffusion-webui/models/Stable-diffusion

  if Download_Base_Model == "V1.5":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt

  elif Download_Base_Model == "V2.1-768":
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O v21.yaml
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt

  else:
    Download_Base_Model == ""
  %cd ..
  %cd ..
  %cd stable-diffusion-webui

  clear_output()
print('[1;90mInstalling SDZ Xformers ✓')

with capture.capture_output() as cap:
  if SDZ_Dreambooth:
    !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
    !bash ./py310.sh -b -f -p /usr/local
    !python -m ipykernel install --name "py310" --user
    !conda install -c "xformers/label/dev" xformers -y

  clear_output()
print('[1;96mInstalling SDZ with Extentions ✓')

with capture.capture_output() as cap:
  else:
    # Install style - config
    os.chdir(SDZ_webui_path)
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml

    # Install Extensions
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git

    os.chdir(SDZ_install+"/"+SDZ)

  clear_output()
print('[1;90mPreparing SDZ System ✓')

if SDZ_Version == "V1.5":
  !python launch.py --share --api --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --no-half
elif SDZ_Version == "V2.1-768":
  !python launch.py --share --api --force-enable-xformers --config="/models/Stable-diffusion/v21.yaml" --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --no-half


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v5</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)

model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/Zv2-1_768-fp16.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = False #@param {type:"boolean"}

with capture.capture_output() as cap:

  if os.path.exists(SDZ):
    shutil.rmtree(SDZ)
  !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  if not os.path.exists(SDZ):
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  %cd stable-diffusion-webui/models/Stable-diffusion
  !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O v21.yaml

  if Download_Base_Model == "V1.5":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

  elif Download_Base_Model == "V2.1":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt

  else:
    Download_Base_Model == ""
  %cd ..
  %cd ..
  %cd stable-diffusion-webui

  clear_output()
print('[1;96mInstalling SDZ ✓')

with capture.capture_output() as cap:
  if SDZ_Dreambooth:
    !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
    !bash ./py310.sh -b -f -p /usr/local
    !python -m ipykernel install --name "py310" --user
    !conda install -c "xformers/label/dev" xformers -y

    # Install Extensions 
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
    

  else:
    # Install style - config

    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml

    # Install Extensions
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git


    os.chdir(SDZ_install+"/"+SDZ)

  clear_output()
print('[1;90mPreparing SDZ System ✓')

if SDZ_Version == "V1.5":
  !python launch.py --share --api --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --no-half
elif SDZ_Version == "V2.1":
  !python launch.py --share --api --force-enable-xformers --config="/models/Stable-diffusion/v21.yaml" --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --no-half


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v6</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)

model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/Zv2-1_768-fp16.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = True #@param {type:"boolean"}

with capture.capture_output() as cap:

  if os.path.exists(SDZ):
    shutil.rmtree(SDZ)
  !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  if not os.path.exists(SDZ):
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  %cd stable-diffusion-webui/models/Stable-diffusion

  if Download_Base_Model == "V1.5":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

  elif Download_Base_Model == "V2.1":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt

  else:
    Download_Base_Model == ""
  %cd ..
  %cd ..
  %cd stable-diffusion-webui

  
  !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O v2-inference-v.yaml
  SDZconfig="--config v2-inference-v.yaml"
  !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

  clear_output()
print('[1;96mInstalling SDZ ✓')

with capture.capture_output() as cap:
  if SDZ_Dreambooth:
    !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
    !bash ./py310.sh -b -f -p /usr/local
    !python -m ipykernel install --name "py310" --user
    !conda install -c "xformers/label/dev" xformers -y

    # Install Extensions 
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
    
  else:
    # Install style - config
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css

    # Install Extensions
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git

    os.chdir(SDZ_webui_path)

  clear_output()
print('[1;90mPreparing SDZ System ✓')

if SDZ_Version == "V1.5":
  !python launch.py --share --api --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --disable-safe-unpickle --no-half
elif SDZ_Version == "V2.1":
  !python launch.py --share --api $SDZconfig --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --disable-safe-unpickle --no-half


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v8</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)

model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/Zv2-1_768-fp16.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = True #@param {type:"boolean"}

with capture.capture_output() as cap:

  if os.path.exists(SDZ):
    shutil.rmtree(SDZ)
  !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  if not os.path.exists(SDZ):
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  %cd stable-diffusion-webui
  !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
  !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
  !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css

  %cd stable-diffusion-webui/models/Stable-diffusion

  if Download_Base_Model == "V1.5":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

  elif Download_Base_Model == "V2.1":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt

  else:
    Download_Base_Model == ""
  %cd ..
  %cd ..
  %cd stable-diffusion-webui
  !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O v2-inference-v.yaml
  SDZconfig="--config v2-inference-v.yaml"
  !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

  %cd extensions
  !git clone https://github.com/Extraltodeus/depthmap2mask.git

  %cd ..

  clear_output()
print('[1;96mInstalling SDZ ✓')

with capture.capture_output() as cap:
  if SDZ_Dreambooth:
    !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
    !bash ./py310.sh -b -f -p /usr/local
    !python -m ipykernel install --name "py310" --user
    !conda install -c "xformers/label/dev" xformers -y

    # Install Extensions 
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
    %cd ..
    
  clear_output()
print('[1;90mPreparing SDZ System ✓')

if SDZ_Version == "V1.5":
  !python launch.py --share --api --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --disable-safe-unpickle --no-half
elif SDZ_Version == "V2.1":
  !python launch.py --share --api $SDZconfig --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --disable-safe-unpickle --no-half


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v9</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)

model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/Zv2-1_768-fp16.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = True #@param {type:"boolean"}

with capture.capture_output() as cap:

  if os.path.exists(SDZ):
    shutil.rmtree(SDZ)
  !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  if not os.path.exists(SDZ):
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  %cd stable-diffusion-webui
  !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
  !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
  !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
  !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
  SDZconfig="--config v2-inference-v.yaml"
  !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

  %cd stable-diffusion-webui/models/Stable-diffusion

  if Download_Base_Model == "V1.5":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

  elif Download_Base_Model == "V2.1":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt

  else:
    Download_Base_Model == ""
  %cd ..
  %cd ..
  %cd stable-diffusion-webui
  %cd extensions

  !git clone https://github.com/Extraltodeus/depthmap2mask.git

  %cd ..

  clear_output()
print('[1;96mInstalling SDZ ✓')

with capture.capture_output() as cap:
  if SDZ_Dreambooth:
    !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
    !bash ./py310.sh -b -f -p /usr/local
    !python -m ipykernel install --name "py310" --user
    !conda install -c "xformers/label/dev" xformers -y

    # Install Extensions 
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
    %cd ..
    
  clear_output()
print('[1;90mPreparing SDZ System ✓')

if SDZ_Version == "V1.5":
  !python launch.py --share --api --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --disable-safe-unpickle --no-half
elif SDZ_Version == "V2.1":
  !python launch.py --share --api $SDZconfig --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --disable-safe-unpickle --no-half


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v10</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"

model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/Zv2-1_768-fp16.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = True #@param {type:"boolean"}

with capture.capture_output() as cap:

  if os.path.exists(SDZ):
    shutil.rmtree(SDZ)
  !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  if not os.path.exists(SDZ):
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

  %cd stable-diffusion-webui
  !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
  !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
  !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
  !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
  !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py
  !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

  %cd stable-diffusion-webui/models/Stable-diffusion

  if Download_Base_Model == "V1.5":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

  elif Download_Base_Model == "V2.1":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt

  else:
    Download_Base_Model == ""
    os.chdir(SDZ_webui_path)
  %cd extensions
  !git clone https://github.com/Extraltodeus/depthmap2mask.git
  %cd ..

  clear_output()
print('[1;96mReady for SDZ ✓')

with capture.capture_output() as cap:
  if SDZ_Dreambooth:
    if not os.path.exists('/usr/local/conda-meta/'):
      !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
      !bash ./py310.sh -b -f -p /usr/local
      !python -m ipykernel install --name "py310" --user
      !conda install -c "xformers/label/dev" xformers -y
    
    if os.path.exists('/usr/local/conda-meta/'):
      # Install Extensions 
      os.chdir(SDZ_webui_path)
      %cd extensions
      !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
      %cd ..
    
  clear_output()
print('[1;90mPreparing SDZ System ✓')

if SDZ_Version == "V1.5":
  !python launch.py --share --api --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --disable-safe-unpickle --no-half
elif SDZ_Version == "V2.1":
  !python launch.py --share --api $SDZconfig --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --cors-allow-origins=https://www.painthua.com --theme=dark --disable-safe-unpickle --no-half


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v11</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"

model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/Zv2-1_768-fp16.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = True #@param {type:"boolean"}
SDZ_Continue = True #@param {type:"boolean"}

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt

    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
    %cd ..

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y
      
      if os.path.exists('/usr/local/conda-meta/'):
        # Install Extensions 
        os.chdir(SDZ_webui_path)
        %cd extensions
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api  --cors-allow-origins=https://www.painthua.com $SDZconfig --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half

  clear_output()
print('[1;90mPreparing SDZ System ✓')
if SDZ_Continue:  
  os.chdir(SDZ_webui_path)
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com $SDZconfig --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v12</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"

model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/Zv2-1_768-fp16.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = True #@param {type:"boolean"}
SDZ_Continue = False #@param {type:"boolean"}

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt

    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
    %cd ..

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
      
      if os.path.exists('/usr/local/conda-meta/'):
        # Install Extensions 
        os.chdir(SDZ_webui_path)
        %cd extensions
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com $SDZconfig --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half

if SDZ_Continue:  
  os.chdir(SDZ_webui_path)
  clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com $SDZconfig --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half


In [ ]:
# fix Dreambooth v21
with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
  filedata = file.read()
filedata = filedata.replace('parameterization: "v"','')
with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
  file.write(filedata)

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/>v12</center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/drive/Shareddrives/Data13/zdream/MyDrive/sdz" #@param {type:"string"}
SDZ_Version = "V2.1" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
gitclo = "git clone --depth 1" 
os.chdir(SDZ_install)
SDZconfig="--config v2-inference-v.yaml"

model_init = "/content/drive/Shareddrives/Data13/ModelzZz/modelz_base/Zv2-1_768-fp16.ckpt" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Dreambooth = True #@param {type:"boolean"}
SDZ_Continue = True #@param {type:"boolean"}

if not SDZ_Continue: 
  with capture.capture_output() as cap:

    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    !wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
    !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py

    %cd stable-diffusion-webui/models/Stable-diffusion

    if Download_Base_Model == "V1.5":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

    elif Download_Base_Model == "V2.1":
      !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
     
      # fix Dreambooth v21
      with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'r') as file :
        filedata = file.read()
      filedata = filedata.replace('parameterization: "v"','')
      with open(SDZ_webui_path+'/'+'v2-inference-v.yaml', 'w') as file:
        file.write(filedata)

    else:
      Download_Base_Model == ""
      os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
    %cd ..

    clear_output()
  print('[1;96mInstalling SDZ ✓')

  with capture.capture_output() as cap:
    if SDZ_Dreambooth:
      if not os.path.exists('/usr/local/conda-meta/'):
        !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
        !bash ./py310.sh -b -f -p /usr/local
        !python -m ipykernel install --name "py310" --user
        !conda install -c "xformers/label/dev" xformers -y
        !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
      
      if os.path.exists('/usr/local/conda-meta/'):
        # Install Extensions 
        os.chdir(SDZ_webui_path)
        %cd extensions
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
        %cd ..
        clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae

if SDZ_Continue:  
  os.chdir(SDZ_webui_path)
  clear_output()
  print('[1;90mPreparing SDZ System ✓')
  if SDZ_Version == "V1.5":
    !python launch.py --share --api --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
  elif SDZ_Version == "V2.1":
    !python launch.py --share --api $SDZconfig --cors-allow-origins=https://www.painthua.com --force-enable-xformers --ckpt-dir=$models_dir --ckpt=$model_init --theme=dark --disable-safe-unpickle --no-half-vae
